# 1. Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
register = pd.read_csv("register.csv")
rooms = pd.read_csv("rooms.csv")

# 2. Cleaning

In [3]:
# Aggregate amounts
grouped = register.groupby(["legal_entity", "service_category", "room_id", "month"])["amount"].sum().reset_index()
joined = pd.merge(grouped, rooms, on="room_id", how="left")
joined.head()

,legal_entity,service_category,room_id,month,amount,building_id,room_area,build_year,building_floors,building_type,building_func,x_coord,y_coord
0,False,Dvinarė kaina,005f6379f5dc0e1,2018-11-01,0.74,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
1,False,Dvinarė kaina,005f6379f5dc0e1,2018-12-01,0.74,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
2,False,Dvinarė kaina,005f6379f5dc0e1,2019-01-01,0.63,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
3,False,Dvinarė kaina,005f6379f5dc0e1,2019-02-01,0.63,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
4,False,Dvinarė kaina,005f6379f5dc0e1,2019-03-01,0.63,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0


In [4]:
joined["amount"].corr(joined["x_coord"])

np.float64(0.0022077555074536473)

In [5]:
dropped = joined.drop(["room_id", "month", "building_id", "building_type"], axis=1)
dropped["build_year"] = pd.to_datetime(dropped["build_year"])
dropped["build_year"] = dropped["build_year"].dt.year
dropped.head()

,legal_entity,service_category,amount,room_area,build_year,building_floors,building_func,x_coord,y_coord
0,False,Dvinarė kaina,0.74,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
1,False,Dvinarė kaina,0.74,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
2,False,Dvinarė kaina,0.63,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
3,False,Dvinarė kaina,0.63,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
4,False,Dvinarė kaina,0.63,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0


In [26]:
filtered = dropped[dropped["service_category"] == "Šiluma"]
len(filtered)

3159820

In [34]:
X, y = filtered.drop(["amount", 'service_category'], axis=1), filtered["amount"]
y

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [36]:
X_train.head(3)

,legal_entity,room_area,build_year,building_floors,building_func,x_coord,y_coord
6749520,False,60.74,1972.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,458077.0,6191479.0
8046888,False,65.11,1982.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,433887.0,6208101.0
7003068,False,64.53,1980.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,453732.0,6198230.0


In [37]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ("imp", SimpleImputer()),
    ("scaler", StandardScaler())
])

int_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
])

ohe_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

ord_pipeline = Pipeline([
    ("ord", OrdinalEncoder()),
    ("imp", SimpleImputer(strategy="most_frequent")),
])

num_cols = ["room_area", "x_coord", "y_coord"]
int_cols = ["build_year", "building_floors"]
ohe_cols = ["building_func"]
ord_cols = ["legal_entity"]

processor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("int", int_pipeline, int_cols),
    ("ohe", ohe_pipeline, ohe_cols),
    ("ord", ord_pipeline, ord_cols)
])

In [38]:
X_train_proc = pd.DataFrame(processor.fit_transform(X_train))
X_test_proc = pd.DataFrame(processor.transform(X_test))

In [40]:
X_test_proc

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.046332,0.194972,-0.355000,1967.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.042753,0.415604,0.409398,1988.0,9.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.035529,1.031013,-0.208603,1991.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.008323,-0.195991,-0.462928,1989.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.009723,-0.279523,-0.613599,1994.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157986,-0.053045,0.655485,0.561494,1968.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157987,-0.039775,0.061867,-0.522056,1977.0,9.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157988,-0.015702,0.477890,0.487405,1967.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157989,-0.031751,0.496230,0.338159,1975.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3. Modelling

In [44]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=10000)
xgb_model.fit(X_train_proc, y_train, eval_set=[(X_test_proc, y_test)], verbose=True)

[0]	validation_0-rmse:3120.50037
[1]	validation_0-rmse:2774.68951
[2]	validation_0-rmse:2569.20074
[3]	validation_0-rmse:2449.09375
[4]	validation_0-rmse:2378.02442
[5]	validation_0-rmse:2325.03871
[6]	validation_0-rmse:2292.52102
[7]	validation_0-rmse:2275.99524
[8]	validation_0-rmse:2263.68296
[9]	validation_0-rmse:2248.37140
[10]	validation_0-rmse:2241.68081
[11]	validation_0-rmse:2232.72535
[12]	validation_0-rmse:2228.01245
[13]	validation_0-rmse:2224.76157
[14]	validation_0-rmse:2217.88436
[15]	validation_0-rmse:2214.61882
[16]	validation_0-rmse:2209.05442
[17]	validation_0-rmse:2209.67771
[18]	validation_0-rmse:2208.52987
[19]	validation_0-rmse:2205.12211
[20]	validation_0-rmse:2203.34335
[21]	validation_0-rmse:2200.78286
[22]	validation_0-rmse:2199.23026
[23]	validation_0-rmse:2198.11283
[24]	validation_0-rmse:2196.51727
[25]	validation_0-rmse:2192.36889
[26]	validation_0-rmse:2191.54241
[27]	validation_0-rmse:2190.94394
[28]	validation_0-rmse:2190.13340
[29]	validation_0-rmse:2

KeyboardInterrupt: 